**Date: 2022-08-20**

**Authors: Asifat Haruna Olawale**

**email: olawaleasifat@gmail.com**

**Copyright © 2022 Asifat Haruna. All rights reserved**

<img src="./images/ETL.PNG" width="500">

<center><h1> Basic Extraction-Transformation-Loading (ETL)  Process</h1></center>

<h4>  This task is divided into two sections: <br> </h4>

* The first section involves fetching data directly from a server using an API and scraping web pages. After obtaining the data, it is subjected to transformation processes before being loaded into a local directory.  

*   The second section demonstrates the fundamental ETL process using diverse locally stored data sources within the directory. Additionally, it demonstrates the utilization of a local Relational Database Management System (RDBMS) for data storage.

# Table of contents

<strong style="color:blue;">SECTION ONE</strong> 
* [1. Extract data using web scraping](#1.-Extract-data-using-web-scraping) 
* [1.1 Transformation](#1.1-Transformation)
* [1.2 Loading into a local directory](#1.2-Loading-into-a-local-directory)

<strong style="color:blue;">SECTION TWO</strong> 
* [2. ETL (Local)](#2.-ETL-(Local))
* [2.1 Extraction ](#2.1-Extraction)
* [2.2. Transformation](#2.2-Transformation)
* [2.3. Loading](#2.3-Loading)
    * [2.3.1 Loading into a local directory](#2.3.1-Loading-into-a-local-directory)
    * [2.3.2 Loading into a local RDMS](#2.3.2-Loading-into-a-local-RDBMS)

<h1 style="text-align:center; color:blue;">SECTION ONE</h1>

## Objectives

The objectives involves :

*   Utilizing web scraping to retrieve the list of the top 100 banks ranked by total assets in USD.
*   Extracting the most recent exchange rate from an API.
*   Subsequently,converting the banks' assets to another currency, specifically GBP.
*   Storing the coverted data into a local directory

## Imports

In [1]:
from bs4 import BeautifulSoup # for parsing HTML content
import requests #requests for making HTTP requests
import pandas as pd  #pandas for data analysis and manipulation
import json #for working with JSON data

## 1. Extract Data Using Web Scraping

Scrape the data related to the largest banks in the world based on market capitalization from the Wikipedia webpage https://en.wikipedia.org/wiki/List_of_largest_banks. Store this data as a DataFrame.

### Webpage Contents

Utilize the `requests` library to retrieve the content of the webpage, and then assign this data to the variable `html_data`

<strong> Data pull request Function<strong/>

In [2]:
def get_html(url):
    with requests.get(url) as response:
        if response.status_code==200:
            html=response.text
            return html
        else:
            raise RuntimeError(response.status_code)
            

html_data=get_html('https://en.wikipedia.org/wiki/List_of_largest_banks')
#html_data

<strong>Scarping the data using BeautifulSoup to parse the retrived contents of the  webpage<strong/>

In [3]:
soup=BeautifulSoup(html_data,'html.parser')

#finds all table in the retrieved webpage
tables=soup.find_all('table')

<div class="alert alert-block alert-info">

<b> Tip: pd.read_html is a Pandas function that reads HTML tables from a web page and returns a list of DataFrame objects containing the table data.  </b>

</div>


In [4]:
table_df=pd.read_html(str(tables), flavor='bs4')

#We can now subset the dataframe list to the dataframe of interest

for id,table in enumerate(table_df):
    columns=table.columns.values=='Bank name'
    if columns.any():
        idx=id
        print('The Table index ',idx ,"has a column 'Bank name' " )
        break 
    else:
        print("None of the DataFrame tables contain the column for 'Bank name'")

The Table index  0 has a column 'Bank name' 


In [5]:
data=table_df[idx]
data.head()

Rank                                        Bank name  \
0     1  Industrial and Commercial Bank of China Limited   
1     2                          China Construction Bank   
2     3                       Agricultural Bank of China   
3     4                                    Bank of China   
4     5                                   JPMorgan Chase   

   Total assets (2022) (US$ billion)  
0                            5742.86  
1                            5016.81  
2                            4919.03  
3                            4192.12  
4                            3665.74

<strong> Alternatively, we could iterate over the tags within the BeautifulSoup object and extract the respective columns.<strong/>

In [6]:
data = pd.DataFrame(columns=["Bank name", "Total assets (2022) (US$ billion)"])

for row in soup.find_all('tbody')[idx].find_all('tr'):
    col = row.find_all('td')
    if col!=[]:
        Bname=col[1].text.strip()
        mcp=col[2].text.strip()
        try:
            mcp=float(str(mcp).replace(",",""))
        except:
            mcp=float(mcp)
        record=pd.DataFrame({"Bank name":[Bname],"Total assets (2022) (US$ billion)":[mcp]})
        data=pd.concat([data,record],axis=0, ignore_index=True)

        

In [7]:
#gives thesame result
data.head()

Bank name  \
0  Industrial and Commercial Bank of China Limited   
1                          China Construction Bank   
2                       Agricultural Bank of China   
3                                    Bank of China   
4                                   JPMorgan Chase   

   Total assets (2022) (US$ billion)  
0                            5742.86  
1                            5016.81  
2                            4919.03  
3                            4192.12  
4                            3665.74

<strong> As part of the cleaning operation the "," in the Total assest colum will be removed</strong>

## Extract Data current rates from an API

<div class="alert alert-block alert-info">
<b>Tip:</b> The registration Page and API key needed to acess this API can be found  in this :<a href="https://apilayer.com/marketplace/exchangerates_data-api">>>> Link </a>
</div>

<div class="alert alert-block alert-warning">
Please ensure that you modify the 'key' variable below with your updated API key.
</div>

In [8]:
key="cypWKtpQQy85ZPdGP40Jky2QKNEMUtX5"
base="USD" #'EUR'
url= f"https://api.apilayer.com/exchangerates_data/latest?base={base}&apikey={key}" 

In [9]:
html=requests.get(url).text
soup=BeautifulSoup(html, 'html.parser')
ex_rate_df=pd.DataFrame(json.loads(str(soup)))

In [10]:
ex_rate_df.head()

success   timestamp base        date       rates
AED     True  1691884444  USD  2023-08-12    3.673042
AFN     True  1691884444  USD  2023-08-12   84.354052
ALL     True  1691884444  USD  2023-08-12   94.669160
AMD     True  1691884444  USD  2023-08-12  388.891422
ANG     True  1691884444  USD  2023-08-12    1.807014

<strong>Conversion rate by currency Function<strong/>

In [11]:
#function return the rates 
def ex_rate(rates_df=None, currency_id=None):
    return(rates_df.loc[currency_id,'rates'])

In [12]:
#for example 
exchange_rate=ex_rate(ex_rate_df,'GBP')
exchange_rate

0.787774

## 1.1 Transformation
[Go back to the "Table of contents"](#Table-of-contents)

Using the sourced API <code>exchange rate dataframe(ex_rate_df)</code> and the <code>exchange_rate  function </code>, convert  the `market capitalization dataframe` from its base currency(USD) to GBP

1.  Changes the `Total assets (year) (US$ billion)` column from USD to GBP
2.  Rounds the Total assets  (US$ Billion)\` column to 3 decimal places
3.  Rename `Total assets (US$ Billion)` to `Total assets  (GBP$ Billion)`


In [13]:
def transform(marketcap_df,ex_rate_df=None,currency_id=None):
    # Write your code here
    assert (isinstance(marketcap_df,pd.DataFrame)), 'The input file must be a dataframe instance'
    assert (isinstance(ex_rate_df,pd.DataFrame)), 'The input file must be a dataframe instance'
    mcap=marketcap_df.copy()
    #apply the rate function
    rate=ex_rate(ex_rate_df,currency_id)
    mcap.iloc[:,1]=mcap.iloc[:,1].apply(lambda x: round(float(x)*rate,3))
    mcap=mcap.rename(columns={mcap.columns.values[1]:f'Total assets (2022) ({currency_id} billion)'})
    return mcap

In [14]:
transformed_data=transform(data,ex_rate_df=ex_rate_df,currency_id='GBP')
transformed_data.head()

Bank name  \
0  Industrial and Commercial Bank of China Limited   
1                          China Construction Bank   
2                       Agricultural Bank of China   
3                                    Bank of China   
4                                   JPMorgan Chase   

   Total assets (2022) (GBP billion)  
0                           4524.076  
1                           3952.112  
2                           3875.084  
3                           3302.443  
4                           2887.775

## 1.2 Loading into a local directory
[Go back to the "Table of contents"](#Table-of-contents)

In [15]:
def load(targetfile=None,data_to_load=None):
    assert(isinstance(data_to_load,pd.DataFrame)), "The data to load must be a class of pandas dataframe"
    data_to_load.to_csv(targetfile, index=False)  

In [16]:
load(targetfile="./transformed_data/Top_100_bank_asset_GBP.csv",data_to_load=transformed_data)

<h1 style="text-align:center; color:blue;">SECTION TWO</h1>

# Importing required python libraries

In [2]:
import glob                         # data access in file manager
import pandas as pd                 # data manipulation
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from os.path import join            # data access in file manager
from datetime import datetime       # for handling datetime
import pint #manipulate or convert physical quantities 
import sqlite3 # is used for interacting with SQLite databases from python

## 2. ETL (Local)
 [Go back to the "Table of contents"](#Table-of-contents)

## Objectives
*   Read CSV,JSON , XML file formats.
*   Extract data from the mentioned file formats.
*   Transform data.
*  Save the transformed data in a ready-to-load format that can be easily loaded into a relational database management system (RDBMS).

## Data description
The dataset has been designed to collect information from various sources, resulting in the inclusion of diverse formats such as structured (.csv) and semi-structured file formats (.xml and .json). It encompasses data related to the height and weight of different individuals.

## 2.1 Extraction
[Go back to the "Table of contents"](#Table-of-contents)

<strong> The following functions extract data from the respective data formats and save it into a dataframe.<strong/>

<strong style="color:blue"> CSV Extract Function<strong/>


In [3]:
def extract_from_csv(files_to_process=[], variables_list=None):
    assert(isinstance(files_to_process, list)), "Please the files_to_process must be a list"
    df_list=[]
    for file in files_to_process:
        dataframe = pd.read_csv(file)[variables_list]
        df_list.append(dataframe)
    if len(df_list)>1:
        dataframe=pd.concat(df_list,ignore_index=True)
    else:
        dataframe=df_list[0]
    return dataframe

<strong style="color:blue"> XML Extract Function<strong/>

In [4]:
def extract_from_xml(files_to_process=[],variables_list=None):
    assert(isinstance(files_to_process, list)), "Please the files_to_process must be a list"
    df_list=[]
    for file in files_to_process:
        tree = ET.parse(file)
        root = tree.getroot()
        tmp_dic={k:[] for k in variables_list}
        for i in root:
            for variable in variables_list:
                tmp = i.find(variable).text
                if tmp.isnumeric():
                    tmp_dic[variable].append(float(tmp))
                else:
                    tmp_dic[variable].append(tmp)
        dataframe=pd.DataFrame(tmp_dic)
        df_list.append(dataframe)
    if len(df_list)>1:
        dataframe=pd.concat(df_list,ignore_index=True)
    else:
        dataframe=df_list[0]
    return dataframe

<strong style="color:blue"> JSON Extract Function<strong/>

In [5]:
def extract_from_json(files_to_process, variables_list=None):
    assert(isinstance(files_to_process, list)), "Please the files_to_process must be a list"
    df_list=[]
    for file in files_to_process:
        dataframe = pd.read_json(file,lines=True)[variables_list]
        df_list.append(dataframe)
    if len(df_list)>1:
        dataframe=pd.concat(df_list,ignore_index=True)
    else:
        dataframe=df_list[0]
    return dataframe

<strong style="color:blue"> Harmonizing all  extracted data<strong/>


In [6]:
def harmonize(source_path=".",variables_list=None, files_extension=["csv"]):
    assert(isinstance(files_extension, list)), "Please the files_extension must be a list"
    func_dic={"xml":extract_from_xml,"csv":extract_from_csv,"json":extract_from_json}
    extracted_list=[]
    for ext in files_extension:
        arg={"files_to_process":glob.glob(join(source_path,f"*.{ext.lower()}")),"variables_list":variables_list} 
        extracted_list.append(func_dic[ext](**arg))
    if len(extracted_list)>1:
        harmonized_df=pd.concat(extracted_list,ignore_index=True)
    else:
        harmonized_df=extracted_list[0]
    return  harmonized_df

In [7]:
extract_from_csv(["./source1.csv","./source2.csv"], variables_list=["name", "height", "weight"])

name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30
5   alex   65.78  112.99
6   ajay   71.52  136.49
7  alice   69.40  153.03
8   ravi   68.22  142.34
9    joe   67.79  144.30

In [8]:
extract_from_xml(["./source2.xml"],variables_list=["name", "height", "weight"])

name height  weight
0  simon  67.90  112.37
1  jacob  66.78  120.67
2  cindy  66.49  127.45
3   ivan  67.62  114.14

In [9]:
extract_from_json(["./source1.json"], variables_list=["name", "height", "weight"])

name  height  weight
0   jack   68.70  123.30
1    tom   69.80  141.49
2  tracy   70.01  136.46
3   john   67.90  112.37

In [10]:
all_data=harmonize(variables_list=["name", "height", "weight"], files_extension=["csv","json","xml"])
all_data[["height","weight"]]=all_data[["height","weight"]].astype(float)
all_data.head()

name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30

## 2.2 Transformation
[Go back to the "Table of contents"](#Table-of-contents)

The transformation function carries out the subsequent actions:

* It converts the height from inches to millimeters.
* It changes the weight from pounds to kilograms.

<div class="alert alert-block alert-info">

<b> Tip:In addition to the transformation of physical units, other transformation processes may include data cleaning and enrichment. These processes encompass tasks such as removing or interpolating missing values and changing the data type. The provided examples offer a straightforward demonstration of the transformation process and assume the dataset has no missing values.  </b>

</div>


In [11]:
def transform(data_df=None, col=None,from_unit=None,to_unit=None):
    sub_df=data_df.copy()
    ureg = pint.UnitRegistry()
    converter=(1 * ureg(from_unit)).to(ureg(to_unit)).magnitude
    if isinstance(col, list):
        for cl in col:
            # Apply the conversion function to a specific column using apply()
            sub_df[cl] = sub_df[cl].apply(lambda x: x*converter)
    else:
        # Apply the function on all of the columns  
        sub_df = sub_df.applymap(lambda x: x*converter)   
    return sub_df


In [12]:
all_data_transformed=transform(all_data,col=["height"],from_unit="inch", to_unit="mm")
all_data_transformed=transform(all_data_transformed,col=["weight"],from_unit="pound", to_unit="kg")

In [13]:
#original data
all_data.head()

name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30

In [14]:
#transformed data
all_data_transformed.head()

name    height     weight
0   alex  1670.812  51.251402
1   ajay  1816.608  61.910823
2  alice  1762.760  69.413240
3   ravi  1732.788  64.564338
4    joe  1721.866  65.453379

## 2.3 Loading
[Go back to the "Table of contents"](#Table-of-contents)

<strong>Once the essential data transformation is complete, the converted data can be stored by saving it locally in a repository, a database, or even in a storage system situated on a remote server or cloud platform. In this demonstration, we will showcase how to store the transformed data in a local repository and also within a local relational database management system (RDMS), specifically using SQLite.<strong/>

## 2.3.1 Loading into a local directory
[Go back to the "Table of contents"](#Table-of-contents)

In [15]:
def load(targetfile=None,data_to_load=None):
    assert(isinstance(data_to_load,pd.DataFrame)), "The data to load must be a class of pandas dataframe"
    data_to_load.to_csv(targetfile, index=False)  

In [16]:
load(targetfile="./transformed_data/ready-to-use-data.csv",data_to_load=all_data_transformed)

## 2.3.2 Loading into a local RDBMS
[Go back to the "Table of contents"](#Table-of-contents)

In [17]:

columns=all_data_transformed.columns.values.tolist()
#add extra column id which is the combination of the names and its index value .
#Note this could any unique value
all_data_transformed['id']=[f"{name}_{id}" for name, id in zip(all_data_transformed['name'].values,all_data_transformed.index.values)]
#reorder columns
columns=["id"]+columns
all_data_transformed=all_data_transformed[columns]
all_data_transformed.head()

id   name    height     weight
0   alex_0   alex  1670.812  51.251402
1   ajay_1   ajay  1816.608  61.910823
2  alice_2  alice  1762.760  69.413240
3   ravi_3   ravi  1732.788  64.564338
4    joe_4    joe  1721.866  65.453379

In [18]:
#create a connection to the database
con=sqlite3.connect("./ETL.db")
#setup a cursor to allow us execute commands on a database
cur=con.cursor()

In [19]:
columns=",".join([cols for cols in all_data_transformed.columns])
create_table_query = f"CREATE TABLE IF NOT EXISTS harmonized_data({columns}, PRIMARY KEY (id))"

In [20]:
# Execute the CREATE TABLE statement
cur.execute(create_table_query)

In [21]:
#populate the harmonized_data table in the ETL database
for rows , cols in all_data_transformed.iterrows():
    cur.execute(f'''INSERT OR IGNORE INTO harmonized_data VALUES {tuple(cols.values)}''')

# Commit the changes and close the connection
con.commit()

In [22]:
#View your data table
for row in cur.execute('''SELECT * FROM harmonized_data'''):
    print(row)


('alex_0', 'alex', 1670.812, 51.25140188630001)
('ajay_1', 'ajay', 1816.6079999999995, 61.91082258130002)
('alice_2', 'alice', 1762.76, 69.41324038110001)
('ravi_3', 'ravi', 1732.7879999999998, 64.56433794580002)
('joe_4', 'joe', 1721.866, 65.45337899100002)
('alex_5', 'alex', 1670.812, 51.25140188630001)
('ajay_6', 'ajay', 1816.6079999999995, 61.91082258130002)
('alice_7', 'alice', 1762.76, 69.41324038110001)
('ravi_8', 'ravi', 1732.7879999999998, 64.56433794580002)
('joe_9', 'joe', 1721.866, 65.45337899100002)
('alex_10', 'alex', 1670.812, 51.25140188630001)
('ajay_11', 'ajay', 1816.6079999999995, 61.91082258130002)
('alice_12', 'alice', 1762.76, 69.41324038110001)
('ravi_13', 'ravi', 1732.7879999999998, 64.56433794580002)
('joe_14', 'joe', 1721.866, 65.45337899100002)
('jack_15', 'jack', 1744.98, 55.92793922100001)
('tom_16', 'tom', 1772.9199999999998, 64.17878443130002)
('tracy_17', 'tracy', 1778.254, 61.89721481020001)
('john_18', 'john', 1724.66, 50.97017461690001)
('jack_19', 'j

In [23]:
#close all operation on the  database
cur.close()
con.close()